# Setting up a station
---

You are given the imports below for all the necessary modules and classes required to set up a station server and its devices for the automation workflow. These include handlers for the YuMi robot, web camera, Tecan XCalibur pump, Ika Plate, and supporting utilities.

In [2]:
from abb_yumi.handler import AbbYuMi
from webcam.handler import Webcam
from tecan_xcalibur.handler import TecanXCalibur
from ika_rct_digital.handler import IkaPlate
from ochra_common.spaces.location import Location
from ochra_common.utils.enum import StationType
from ochra_manager.station.station_server import StationServer

## Construct station server
In OChRA framework, a station represents a physical location in the lab that have a number of instruments and robots attached to it that users can operate to run their workflows.

In the code box below, construct a `StationServer` using the following information:
- Name the station *yumi_station*
- The station is placed at yumi_corner, which is located in the ACL lab at main room
- The station server is to run at the address *127.0.0.1:8002*
- Station logs should be stored at the path /tmp/yumi_station

In [3]:
# construct the station server
yumi_station = StationServer(
    name="yumi_station",
    location=Location(lab="ACL", room="main", place="yumi_corner"),
    station_type=StationType.WORK_STATION,
    station_ip="127.0.0.1",
    station_port=8002,
    logging_path="/tmp/yumi_station",
)

## Connect to lab server
In order to add a station to the lab, it needs to connect to the lab server. Using `setup` method, connect to the lab server whose address: *127.0.0.1:8001* 

In [4]:
# setup the station server
yumi_station.setup(lab_ip="127.0.0.1:8001")

[ERROR - 2025-09-24 14:37:32,634][ochra_common.connections.lab_connection]: HTTPConnectionPool(host='127.0.0.1', port=8001): Max retries exceeded with url: /stations/construct (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022B2DAA2180>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


LabEngineException: Request Failed: HTTPConnectionPool(host='127.0.0.1', port=8001): Max retries exceeded with url: /stations/construct (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022B2DAA2180>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

## Setup devices
After constructing the station server and connecting to the lab server, we need to add our instruments and robots to the station. In the code box below, construct and add the following devices to the station:
- **IKA RCT Digital plate** for heating and stirring samples
- **Tecan XCalibur syringe pump** for liquid handling
- **An ordinary web camera** to take pictures/videos of samples
- **ABB YuMi dual-arms robot** to manipulate samples 

In [ ]:
# Construct devices

# construct IKA plate
ika_plate = IkaPlate(name="ika_plate")

# construct Tecan XCalibur pump with reagents map {"water": 1}
tecan_pump = TecanXCalibur(name="tecan_xcalibur_pump", reagents_map={"water": 1})

# construct web camera on usb port ttyUSB0
camera = Webcam(name="camera", usb_port="ttyUSB0")

# construct YuMi robot
# here are the tasks that the robot can perform in this station
tasks = [
    "pick_vial_from_input_loc",
    "decap",
    "place_vial_in_pump",
    "pick_vial_from_pump",
    "place_vial_in_ika",
    "pick_vial_from_ika",
    "pick_vial_from_camera",
    "place_vial_in_camera",
    "cap",
    "place_vial_in_output_loc",
]
yumi = AbbYuMi(name="yumi", available_tasks=tasks)

# add all devices to the station server
yumi_station.add_device(ika_plate)
yumi_station.add_device(tecan_pump)
yumi_station.add_device(camera)
yumi_station.add_device(yumi)

## Run the station server
After adding all the available devices, we need to run the station server in order for it to be ready to handle requests. In the code box below use the `run` command to start the station server

In [ ]:

# start the station server
yumi_station.run()
